In [1]:
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data = pd.read_csv('E:\\data\\sip\\after_LOF.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39555 entries, 0 to 39554
Data columns (total 28 columns):
Unnamed: 0                   39555 non-null int64
SCREEN_RESOLUTION_LONG       39555 non-null float64
SCREEN_RESOLUTION_WIDTH      39555 non-null float64
VIDEO_CLARITY                39555 non-null int64
VIDEO_BITRATE                39555 non-null int64
INITIAL_DELAY                39555 non-null int64
INITIAL_DELAY_PEAK_RATE      39555 non-null float64
VIDEO_STALLING_NUM           39555 non-null int64
VIDEO_STALLING_TOTAL_TIME    39555 non-null int64
VIDEO_TOTAL_TIME             39555 non-null float64
VIDEO_PLAY_TOTAL_TIME        39555 non-null int64
VIDEO_ALL_PEAK_RATE          39555 non-null int64
VIDEO_AVERAGE_RATE           39555 non-null int64
PHONE_ELECTRIC_START         39555 non-null int64
PHONE_ELECTRIC_END           39555 non-null int64
PHONE_VERSION                39555 non-null int64
PING_AVG_RTT                 39555 non-null float64
USER_SCORE                   39

In [4]:
data = pd.read_csv('E:\\data\\sip\\feature_processing.csv')
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
data = data.dropna()
data = data.reset_index(drop = True)

In [15]:
def lof(data, predict=None, k=5, method=1, plot=False):
    
    # 判断是否传入测试数据，若没有传入则测试数据赋值为训练数据
    try:
        if predict == None:
            predict = data.copy()
    except Exception:
        pass
    predict = pd.DataFrame(predict)
    # 计算 LOF 离群因子
    clf = LocalOutlierFactor(n_neighbors=k + 1, algorithm='auto', contamination=0.1, n_jobs=-1)
    clf.fit(data)
    # 记录 k 邻域距离
    predict['k distances'] = clf.kneighbors(predict)[0].max(axis=1)
    # 记录 LOF 离群因子，做相反数处理
    predict['local outlier factor'] = -clf._decision_function(predict.iloc[:, :-1])
    # 根据阈值划分离群点与正常点
    outliers = predict[predict['local outlier factor'] > method].sort_values(by='local outlier factor')
    inliers = predict[predict['local outlier factor'] <= method].sort_values(by='local outlier factor')
    return outliers, inliers

In [16]:
outliers, inliers = lof(data)

In [18]:
data_1 = outliers.reset_index(drop = True)

In [27]:
data_1 = data_1[data_1['EVMOS'] != 0]

In [29]:
data_1 = data_1[data_1['USER_SCORE'] != 0]

In [30]:
data_1 = data_1[data_1['ELOADING'] != 0]
data_1 = data_1[data_1['ESTALLING'] != 0]

In [31]:
data_1['ESTALLING'].value_counts()

5.0    30446
4.0     4760
3.0     2552
1.0      917
2.0      880
Name: ESTALLING, dtype: int64

In [32]:
data_1['ELOADING'].value_counts()

5.0    28289
4.0     5729
3.0     3325
2.0     1164
1.0     1048
Name: ELOADING, dtype: int64

In [33]:
data_1['USER_SCORE'].value_counts()

5.0    28880
4.0     5538
3.0     3347
2.0      958
1.0      832
Name: USER_SCORE, dtype: int64

In [35]:
data_1['EVMOS'].value_counts()

5.0    27871
4.0     6395
3.0     3571
2.0      863
1.0      855
Name: EVMOS, dtype: int64

In [36]:
data_1.to_csv('E:\\data\\sip\\after_LOF.csv')